In [33]:
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute, JSONAttribute, BinaryAttribute
import json, pickle, logging, hashlib, os, zlib, lzma
from datetime import datetime
class Cacher(Model):
  """
  cache for function
  """
  cacheKey = UnicodeAttribute(hash_key=True)
  data = JSONAttribute(default={})
  compressedData = BinaryAttribute(null=True)
  timestamp = NumberAttribute()
  def __repr__(self):
    return json.dumps({
        'cacheKey': self.cacheKey,
        'data': self.data,
        'timestamp': self.timestamp
                })

  @staticmethod
  def hashValue(inputDict:dict):
      return hashlib.sha256(json.dumps(inputDict,sort_keys=True).encode()).hexdigest()

  @classmethod
  def getCache(cls, input, timeout = 86400, verbose=False, compression = True):
    # check cache for value
    cache = next(cls.query(cls.hashValue(input)), None)
    if cache and (datetime.now().timestamp() - cache.timestamp < timeout):
      logging.debug('log found')
      if not compression:
        return cache.data
      try:
        return cls.decompress(cache.compressedData)
      except:
        logging.exception('error decompressiong, perhaps data is not compressed?')
        return cache.data
    else:
      logging.warning('cache not found or expired')
      return None
    
  @classmethod
  def addCache(cls, input:dict, output:dict, compression = True):
    cache = cls(
        cacheKey = cls.hashValue(input),
        data = output if not compression else {},
        timestamp = datetime.now().timestamp(),
        compressedData = cls.compress(input) if compression else None
    )
    try:
        return cache.save()
    except Exception as e:
        logging.exception(f'{e}')
  @staticmethod
  def compress(inputDict:dict,method = zlib)->bin:
    return zlib.compress(json.dumps(inputDict).encode())
  @staticmethod
  def decompress(data:bin, method = zlib)->dict:
    return json.loads(zlib.decompress(data).decode())



In [34]:
# %%timeit
def testmethod(method):
  %timeit compressed = Cacher.compress([i for i in range(100000)],method = method)
  compressed = Cacher.compress([i for i in range(100000)], method = method)
  %timeit decompressed = Cacher.decompress(compressed, method = method)
  decompressed = Cacher.decompress(compressed, method = method)
# decompressed

In [35]:
# testmethod(lz4.frame)
# testmethod(zlib)
# testmethod(gzip)
# testmethod(lzma)

# Test

In [36]:
# #hide
# import pickle
# credLocation = '/Users/nic/.dynamoCache'
# user =''
# pw = ''
# if user and pw:
#   with open (credLocation , 'wb') as f:
#     pickle.dump({
#         'user': user,
#         'pw': pw
#     }, f)
# with open(credLocation , 'rb') as f:
#   creden = pickle.load(f)
#   PW = creden['pw']
#   USER = creden['user']
# # print(PW, USER)

In [53]:
class Cache(Cacher):
  class Meta:
      table_name = 'dynamoCacheTest'
      region = 'ap-southeast-1'
      dax_read_endpoints = ['longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111'] # optional dax, note that dax will speed up dynamodb significantly
      dax_write_endpoints = ['longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111']# optional dax, note that dax will speed 
      billing_mode= 'PAY_PER_REQUEST'

In [54]:
Cache.create_table(billing_mode="PAY_PER_REQUEST")

In [55]:
# %time
Cache.addCache({'hello':'world'}, {'output': 'dummy'}, compression = False)

{'ConsumedCapacity': {'TableName': 'dynamoCacheTest', 'CapacityUnits': 1.0}}

In [56]:
Cache.getCache({'hello':'world'}, compression = False)

{'output': 'dummy'}